In [116]:
import json, os,sys
sys.path.append(os.path.abspath(".")) 
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("aside"))
import torch
from utils.utils import *
from utils.model_utils import load_model
from functools import partial
import ast
import copy
home_dir = "/export/home2/weijie210/ipi_huawei"
from vllm import LLM, SamplingParams
from tqdm import tqdm
from datasets import Dataset
os.environ["TOKENIZERS_PARALLELISM"] = "false"

INFO 09-21 00:35:42 __init__.py:190] Automatically detected platform cuda.
[2025-09-21 00:35:46,526] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/export/home2/weijie210/miniconda3/envs/cot/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/export/home2/weijie210/miniconda3/envs/cot/compiler_compat/ld: warning: libstdc++.so.6, needed by /cm/shared/apps/cuda12.1/toolkit/12.1.1/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/export/home2/weijie210/miniconda3/envs/cot/compiler_compat/ld: warning: libm.so.6, needed by /cm/shared/apps/cuda12.1/toolkit/12.1.1/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/export/home2/weijie210/miniconda3/envs/cot/compiler_compat/ld: /cm/shared/apps/cuda12.1/toolkit/12.1.1/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/export/home2/weijie210/miniconda3/envs/cot/compiler_compat/ld: /cm/shared/apps/cuda12.1/toolkit/12.1.1/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/export/home2/weijie210/miniconda3/envs/cot/compiler_compat/ld: /cm/

In [4]:
model_path = 'Qwen/Qwen3-8B'
model,tokenizer,is_aside = load_model(model_path,use_vllm=True,dtype=torch.bfloat16,vllm_kwargs = {'gpu_memory_utilization':0.8,'enable_chunked_prefill':False})

INFO 09-21 00:36:00 config.py:542] This model supports multiple tasks: {'embed', 'classify', 'reward', 'generate', 'score'}. Defaulting to 'generate'.
INFO 09-21 00:36:00 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='Qwen/Qwen3-8B', speculative_config=None, tokenizer='Qwen/Qwen3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=Qwen/Qwen3-8B, num_scheduler_steps=1, multi_step_stream_outputs=True

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


INFO 09-21 00:36:07 model_runner.py:1115] Loading model weights took 15.2585 GB
INFO 09-21 00:36:09 worker.py:267] Memory profiling takes 2.18 seconds
INFO 09-21 00:36:09 worker.py:267] the current vLLM instance can use total_gpu_memory (79.33GiB) x gpu_memory_utilization (0.80) = 63.46GiB
INFO 09-21 00:36:09 worker.py:267] model weights take 15.26GiB; non_torch_memory takes 0.14GiB; PyTorch activation peak memory takes 5.30GiB; the rest of the memory reserved for KV Cache is 42.76GiB.
INFO 09-21 00:36:09 executor_base.py:110] # CUDA blocks: 19462, # CPU blocks: 1820
INFO 09-21 00:36:09 executor_base.py:115] Maximum concurrency for 32768 tokens per request: 9.50x
INFO 09-21 00:36:10 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_

Capturing CUDA graph shapes: 100%|███████████████████████████████████████████████████████████████████████| 35/35 [00:19<00:00,  1.83it/s]

INFO 09-21 00:36:29 model_runner.py:1562] Graph capturing finished in 19 secs, took 4.77 GiB
INFO 09-21 00:36:29 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 22.83 seconds


In [13]:
toolbench_ds = json.load(open(f"/export/home2/weijie210/ToolBench/ToolBench_data/data/toolllama_G123_dfs_eval.json","r"))
print (len(toolbench_ds))

762


In [14]:
def parse_assistant_response(response):
    action_input = None
    action = None
    thought = None
    if "Action Input:" in response:
        action_input = response.split("Action Input:")[-1].strip().replace('\n',' ')
        try:
            action_input = json.loads(action_input)
        except:
            try:
                action_input = ast.literal_eval(action_input)
            except:
                action_input = None
        if action_input is not None:
            if "Action:" in response:
                action = response.split("Action:")[-1].split("Action Input:")[0].strip()
                thought = response.split("Thought:")[-1].split("Action:")[0].strip()
    elif "Action:" in response:
        action = response.split("Action:")[-1].strip()
        thought = response.split("Thought:")[-1].split("Action:")[0].strip()
    return thought, action, action_input

def convert_turn_keys(messages):
    messages = copy.deepcopy(messages)
    for m in messages:
        m['role'] = m.pop('from')
        m['content'] = m.pop('value')
        if m['role'] == 'function':
            m['role'] = 'tool'
    return messages

def format_assistant_response(action,action_input):
    out_format = "<tool_call>\n{tool_call}\n</tool_call>"
    tool_call = str({
        'name': action,
        'arguments': action_input
    }).replace("'",'"')
    return out_format.format(tool_call = tool_call)

def process_tool_response(resp):
    str_resp =  resp.replace('<tool_call>','').replace('</tool_call>','').strip()
    try:
        tool_call = json.loads(str_resp)
    except:
        try:
            tool_call = ast.literal_eval(str_resp)
        except:
            tool_call = None
    return tool_call
        
sampling_params = SamplingParams(temperature=0., max_tokens=100,)

In [15]:
all_trained_samples = []
for sample in toolbench_ds[:100000]:
    tools = []
    sample = copy.deepcopy(sample) # avoid any potential in-place modification
    trained_sample = []
    assert sample['conversations'][0]['from'] == 'system'
    for j,d in enumerate(sample['conversations']):
        if d['from'] == 'system':
            system_prompt = d['value'] # get the tools
            tools = system_prompt.split('you have access to the following APIs:')[-1].strip()
            tools = ast.literal_eval(tools)
            if not isinstance(tools, list) and not all([isinstance(t, dict) for t in tools]): # cant parse properly.
                break
            tools = [t for t in tools if t['name'].lower() != 'finish'] 
        
        if d['from'] == 'assistant':
            thought, action, action_input = parse_assistant_response(d['value'])
            if any([x is None for x in [action, action_input]]): # parsing failed
                break
            d['value'] = format_assistant_response(action,action_input) #
            if action.lower().strip() =='finish': # not a tool, just the string
                if 'final_answer' not in action_input: # end of turn fail since no final answer
                    break
                sample['conversations'][j]['value'] = action_input['final_answer']
            elif process_tool_response(sample['conversations'][j]['value']) is None: # parsing failed
                continue
            if any([sample['conversations'][k]['from'] == 'function' for k in range(j)]): # we only start collecting after the first tool response (this is for aside, which requires rotating on the function space)
                trained_sample.append(
                { 
                'input': convert_turn_keys(sample['conversations'][1:j]), # this the input (take away the system prompt)
                'label': sample['conversations'][j]['value'],
                'tools': tools
                }
            )
    if len(trained_sample) > 0:
        all_trained_samples.append(np.random.choice(trained_sample)) # randomly choose one turn to train on
print (len(all_trained_samples))
                

600


In [29]:
import concurrent.futures
sampling_kwargs = {'max_tokens':512,'temperature':0.}
def select_label(messages,model,tokenizer,is_tool=True):
    """
    For each sample, generate a response.
    If is tool calling, check if the tool name and args is correct. do literal string match after json.load and dump
    """
    to_train = []
    formatted = [tool_prompt_format(d['input'],tools=d['tools'],tokenizer=tokenizer,enable_thinking=False) for d in messages]
    resps = vllm_generate(model,formatted,sampling_params)
    for resp,d in zip(resps,messages):
        label = process_tool_response(d['label'])
        processed_resp = process_tool_response(resp)
        chk_condition = lambda x,y: True if not is_tool else lambda x,y: str(x).strip() != str(y).strip()
        if chk_condition(label, processed_resp): # we have a sample to train
            to_train.append({
                'prompt' : d['input'],
                'chosen': d['label'],
                'rejected': resp,
                'tools': d['tools']
            })
    return to_train

def compute_length(sample):
    text = tool_prompt_format(
        sample["input"],
        tools=sample["tools"],
        tokenizer=tokenizer,
        enable_thinking=False
    )
    length = len(tokenizer.encode(text, add_special_tokens=True))
    sample["input_len"] = length
    return sample

results = async_process(compute_length, all_trained_samples, workers=32, msg="Computing input lengths")
    
MAX_LENGTH = 4096 - 512

dpo_input_ds = [d for d in results if d["input_len"] < MAX_LENGTH]
tool_dpo_input_ds= [d for d in dpo_input_ds if "<tool_call>" in d["label"]]

print(f"After filtering, we have {len(dpo_input_ds)} samples to finetune on.")
print(f"Total tool calling samples: {len(tool_dpo_input_ds)} out of {len(dpo_input_ds)}")
    

Computing input lengths: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:00<00:00, 2678.73it/s]

After filtering, we have 578 samples to finetune on.
Total tool calling samples: 505 out of 578


In [ ]:
finish_dpo_ds = [d for d in dpo_input_ds if "<tool_call>" not in d["label"]]
print(f"Total finish calling samples: {len(finish_dpo_ds)} out of {len(dpo_input_ds)}")
batch_size = 1024
dpo_ds = []
for i in tqdm(range(0, len(tool_dpo_input_ds), batch_size)):
    batch = dpo_input_ds[i:i+batch_size]
    dpo_ds.extend(select_label(batch,model,tokenizer,is_tool=True))
print (f"Total DPO tool samples: {len(dpo_ds)} out of {len(dpo_input_ds)}")

end_dpo_ds = []
for i in tqdm(range(0, len(finish_dpo_ds), batch_size)):
    batch = finish_dpo_ds[i:i+batch_size]
    end_dpo_ds.extend(select_label(batch,model,tokenizer,is_tool=True))
print (f"Total Ending DPO tool samples: {len(end_dpo_ds)} out of {len(finish_dpo_ds)}")

full_dpo_ds = dpo_ds + end_dpo_ds
print (f"Total DPO samples: {len(full_dpo_ds)}")

permutation_ids = np.random.permutation(len(full_dpo_ds))
full_dpo_ds = [full_dpo_ds[i] for i in permutation_ids]
with open("/export/home2/weijie210/ipi_huawei/aside/data/train_data/dpo_eval.json","w") as f:
    json.dump(full_dpo_ds,f,indent=4)

Total finish calling samples: 73 out of 578


100%|██████████| 1/1 [02:27<00:00, 147.20s/it]


Total DPO tool samples: 510 out of 578
Total DPO samples: 583


In [44]:
with open("/export/home2/weijie210/ipi_huawei/aside/data/train_data/dpo_train.json","r") as f:
    eval_ds = json.load(f)


In [58]:
def convert_str_to_list(examples):
    for e in examples:
        e['chosen'] = [{'role':'assistant','content':e['chosen']}]
        e['rejected'] = [{'role':'assistant','content':e['rejected']}]
    return examples
def apply_chat_tool_template(example,tokenizer):
    prompt = tokenizer.apply_chat_template(example["prompt"], tools=example['tools'], tokenize=False, add_generation_prompt=True)
    chosen = tokenizer.apply_chat_template(example["prompt"] + example['chosen'],tools=example['tools'], tokenize=False, add_generation_prompt=False)
    chosen = chosen[len(prompt):]
    rejected = tokenizer.apply_chat_template(example["prompt"] + example['rejected'],tools=example['tools'], tokenize=False, add_generation_prompt=False)
    rejected = rejected[len(prompt):]
    return {"prompt": prompt, "chosen": chosen, "rejected": rejected}

In [59]:
invalid = 0
formatted_eval_ds = convert_str_to_list(eval_ds)
formatted_eval_ds = [apply_chat_tool_template(d,tokenizer) for d in formatted_eval_ds]
    
